# Finetune HF Llama 3.0 and Deploy on AWS Bedrock

This notebook has the following steps: 

1. imports and converts [Llama 3.0 8b](https://huggingface.co/meta-llama/Meta-Llama-3-8B) from Hugging Face transformer file format to .nemo file format

    Note: you will need to create a HuggingFace account and request access to the model

2. Supervised Fine Tuning (SFT) using the NeMo framework on the [NVIDIA Daring-Anteater dataset](https://huggingface.co/datasets/nvidia/Daring-Anteater), a comprehensive dataset for instruction tuning

3. Move your finetuned model to AWS S3 for use with AWS Bedrock Custom Model Import

## Convert Hugging Face Model to NeMo

In [ ]:
!pip install ipywidgets

In [ ]:
import os
import huggingface_hub

# Set your Hugging Face access token
huggingface_hub.login("HUGGING_FACE_TOKEN")
os.makedirs("/demo-workspace/Meta-Llama-3-8B" ,exist_ok=True)
huggingface_hub.snapshot_download(repo_id="meta-llama/Meta-Llama-3-8B", repo_type="model", local_dir="Meta-Llama-3-8B")

In [ ]:
%%bash
# clear any previous temporary weights dir if any
rm -r model_weights

#converter script from NeMo
python /opt/NeMo/scripts/checkpoint_converters/convert_llama_hf_to_nemo.py \
  --precision bf16 \
  --input_name_or_path=/demo-workspace/Meta-Llama-3-8B \
  --output_path=/demo-workspace/Meta-Llama-3-8B.nemo \

## Import and Configure Dataset

In [ ]:
from datasets import load_dataset
import json

dataset = load_dataset("nvidia/daring-anteater")

for split, shard in dataset.items():
    length = len(shard)
    train_limit = length * 0.85
    with open("daring-anteater-train.jsonl", "w") as train:
        with open("daring-anteater-val.jsonl", "w") as val:
            for count, line in enumerate(shard):
                desired_data = {
                    "system": line["system"],
                    "conversations": line["conversations"],
                    "mask": line["mask"],
                    "type": "TEXT_TO_VALUE",
                }
                if count < train_limit:
                    json.dump(desired_data, train)
                    train.write('\n')
                else:
                    json.dump(desired_data, val)
                    val.write('\n')

## Finetuning

In [ ]:
%%bash

# Set paths to the model, train, validation and test sets.
MODEL="/demo-workspace/Meta-Llama-3-8B.nemo"
TRAIN_DS="./daring-anteater-train.jsonl"
VALID_DS="./daring-anteater-val.jsonl"
TEST_DS="./daring-anteater-val.jsonl"
TEST_NAMES="[daring-anteater]"

SCHEME="none"  # SFT is none
TP_SIZE=2
PP_SIZE=1

OUTPUT_DIR="/demo-workspace/llama3-8b-daring-anteater-sft-3"

export HYDRA_FULL_ERROR=1

python /opt/NeMo-Aligner/examples/nlp/gpt/train_gpt_sft.py \
   trainer.precision=bf16 \
   trainer.num_nodes=1 \
   trainer.devices=8 \
   trainer.sft.max_steps=-1 \
   trainer.sft.limit_val_batches=40 \
   trainer.sft.val_check_interval=1000 \
   model.megatron_amp_O2=True \
   model.restore_from_path=${MODEL} \
   model.optim.lr=5e-6 \
   model.tensor_model_parallel_size=${TP_SIZE} \
   model.pipeline_model_parallel_size=${PP_SIZE} \
   model.data.chat=True \
   model.data.num_workers=0 \
   model.data.train_ds.micro_batch_size=1 \
   model.data.train_ds.global_batch_size=4 \
   model.data.train_ds.max_seq_length=8192 \
   model.data.train_ds.file_path=${TRAIN_DS} \
   model.data.validation_ds.micro_batch_size=1 \
   model.data.validation_ds.global_batch_size=4 \
   model.data.validation_ds.file_path=${VALID_DS} \
   model.data.validation_ds.max_seq_length=8192 \
   exp_manager.create_wandb_logger=False \
   exp_manager.explicit_log_dir=${OUTPUT_DIR} \
   exp_manager.wandb_logger_kwargs.project=llama3-8b-sft \
   exp_manager.wandb_logger_kwargs.name=chat_sft_run \
   exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True \
   exp_manager.resume_if_exists=False \
   exp_manager.resume_ignore_no_checkpoint=True \
   exp_manager.create_checkpoint_callback=True \
   exp_manager.checkpoint_callback_params.monitor=val_loss

## Import Model to AWS S3

To prepare the model for use with BedRock, we must first convert our finetuned model weights back to HF safetensors. The model and the original llama 3.0 tokens will then be sent to your S3 bucket. 

In [ ]:
%%bash
python /opt/NeMo/scripts/checkpoint_converters/convert_llama_nemo_to_hf.py \
--input_name_or_path /demo-workspace/llama3-8b-daring-anteater-sft-3/checkpoints/megatron_gpt_sft.nemo \
--output_path /demo-workspace/llama-output-weights.bin \
--hf_input_path /demo-workspace/Meta-Llama-3-8B \
--hf_output_path /demo-workspace/sft-llama-3-hf

In [ ]:
%%bash

export AWS_ACCESS_KEY_ID=<INSERT_ACCESS_KEY_ID>
export AWS_SECRET_ACCESS_KEY=<INSERT_SECRET_ACCESS_KEY>

./s5cmd cp /demo-workspace/sft-llama-3-hf s3://<INSERT_BUCKET_NAME>

./s5cmd cp /demo-workspace/Meta-Llama-3.0-8B/tokenizer.json s3://<INSERT_BUCKET_NAME>/sft-llama-3-hf/
./s5cmd cp /demo-workspace/Meta-Llama-3.0-8B/tokenizer_config.json s3://<INSERT_BUCKET_NAME>/sft-llama-3-hf/
./s5cmd cp /demo-workspace/Meta-Llama-3.0-8B/original/tokenizer.model s3://<INSERT_BUCKET_NAME>/sft-llama-3-hf/

To run with BedRock, go to the Custom Model import feature and load your model from your S3 bucket. Once the model is ready, it can directly be used for your production inference. 